# Rubiks Cube Solver

The first thing were going to do is initalize our rubiks cube.

In [1]:
from cube import RubiksCube

cube = RubiksCube(n=3)
cube.show()

                 ['w', 'w', 'w']
                 ['w', 'w', 'w']
                 ['w', 'w', 'w']

['o', 'o', 'o']  ['g', 'g', 'g']  ['r', 'r', 'r']  ['b', 'b', 'b']
['o', 'o', 'o']  ['g', 'g', 'g']  ['r', 'r', 'r']  ['b', 'b', 'b']
['o', 'o', 'o']  ['g', 'g', 'g']  ['r', 'r', 'r']  ['b', 'b', 'b']

                 ['y', 'y', 'y']
                 ['y', 'y', 'y']
                 ['y', 'y', 'y']


Now that we have the rubiks cube we can shuffle it to a random state.

In [2]:
#cube.shuffle()
cube.horizontal_twist(1, 0)
cube.vertical_twist(1, 0)
cube.side_twist(0, 1)
cube.horizontal_twist(0, 1)
cube.show()

                 ['o', 'w', 'w']
                 ['g', 'o', 'b']
                 ['o', 'w', 'w']

['b', 'o', 'b']  ['y', 'o', 'o']  ['g', 'w', 'g']  ['r', 'r', 'w']
['g', 'g', 'g']  ['r', 'w', 'r']  ['b', 'b', 'b']  ['y', 'y', 'y']
['y', 'o', 'o']  ['g', 'w', 'g']  ['r', 'r', 'w']  ['b', 'o', 'b']

                 ['y', 'g', 'y']
                 ['y', 'r', 'y']
                 ['r', 'b', 'r']


In [3]:
from random import choice

class IDA_star(object):
    def __init__(self, heuristic, colour = 'w', max_depth = 20):
        self.max_depth = max_depth
        self.colour = colour
        self.threshold = max_depth
        self.min_threshold = None
        self.heuristic = heuristic
        self.moves = []
    
    def run(self, state):
        while True:
            status = self.search(state, 1)
            if status: return self.moves
            self.moves = []
            self.threshold = self.min_threshold
            #print(self.threshold)
            #print('----')
        return []

    def search(self, state, g_score):
        cube = RubiksCube(state=state)
        #target_face = find_target_face(cube, self.colour)
        #if cube.solved() or cross_solved(cube.cube, target_face, colour=self.colour):
        if cube.solved():
            return True
        elif len(self.moves) >= self.threshold:
            return False
        min_val = float('inf')
        best_action = None
        for a in [(r, n, d) for r in ['h', 'v', 's'] for d in [0, 1] for n in range(cube.n)]:
            cube = RubiksCube(state=state)
            if a[0] == 'h':
                cube.horizontal_twist(a[1], a[2])
            elif a[0] == 'v':
                cube.vertical_twist(a[1], a[2])
            elif a[0] == 's':
                cube.side_twist(a[1], a[2])
            if cube.solved():
                self.moves.append(a)
                return True
            cube_str = cube.stringify()
            h_score = self.heuristic[cube_str] if cube_str in self.heuristic else self.max_depth
            f_score = g_score + h_score
            if f_score < min_val:
                min_val = f_score
                best_action = [(cube_str, a)]
            elif f_score == min_val:
                if best_action is None:
                    best_action = [(cube_str, a)]
                else:
                    best_action.append((cube_str, a))
        if best_action is not None:
            if self.min_threshold is None or min_val < self.min_threshold:
                self.min_threshold = min_val
            next_action = choice(best_action)
            self.moves.append(next_action[1])
            status = self.search(next_action[0], g_score + min_val)
            if status: return status
        return False
        

In [4]:
def build_heuristic_db(state, actions, max_moves = 20, heuristic = None):
    if heuristic is None:
        heuristic = {state: 0}
    que = [(state, 0)]
    while True:
        if not que:
            break
        s, d = que.pop()
        if d > max_moves:
            continue
        for a in actions:
            cube = RubiksCube(state=s)
            if a[0] == 'h':
                cube.horizontal_twist(a[1], a[2])
            elif a[0] == 'v':
                cube.vertical_twist(a[1], a[2])
            elif a[0] == 's':
                cube.side_twist(a[1], a[2])
            a_str = cube.stringify()
            if a_str not in heuristic or heuristic[a_str] > d + 1:   
                heuristic[a_str] = d + 1
            que.append((a_str, d+1))
    return heuristic

In [ ]:
import json
import os.path

if os.path.exists('heuristic.json'):
    with open('heuristic.json') as f:
        h_db = json.load(f)
else:
    h_db = None

cube = RubiksCube(n=3)
actions = [(r, n, d) for r in ['h', 'v', 's'] for d in [0, 1] for n in range(cube.n)]
h_db = build_heuristic_db(
    cube.stringify(), 
    actions, 
    max_moves = 5, 
    heuristic = h_db
)

with open('heuristic.json', 'w', encoding='utf-8') as f:
    json.dump(
        h_db,
        f,
        ensure_ascii=False,
        indent=4
    )

In [ ]:
cube.horizontal_twist(1, 0)
cube.vertical_twist(1, 0)
cube.side_twist(0, 1)
cube.horizontal_twist(0, 1)
cube.horizontal_twist(1, 0)
cube.vertical_twist(1, 0)
cube.horizontal_twist(1, 0)

solver = IDA_star(h_db)
moves = solver.run(cube.stringify())
print(moves)

for m in moves:
    if m[0] == 'h':
        cube.horizontal_twist(m[1], m[2])
    elif m[0] == 'v':
        cube.vertical_twist(m[1], m[2])
    elif m[0] == 's':
        cube.side_twist(m[1], m[2])
cube.show()

In [ ]:
def find_target_face(cube, colour):
    target_face = None #Target face position (Middle cube of target colour)
    for i, face in enumerate(cube.cube):
        if face[1][1] == colour:
            target_face = i
            break
    return target_face

In [ ]:
def target_crossed(cube, target, colour='w'):
    if cube[target][0][1] == colour and cube[target][1][0] == colour and cube[target][1][1] == colour and cube[target][1][2] == colour and cube[target][2][1] == colour:
        return True
    return False

In [ ]:
def cross_solved(cube, target, colour='w'):
    if cube[target][0][1] == colour and cube[target][1][0] == colour and cube[target][1][1] == colour and cube[target][1][2] == colour and cube[target][2][1] == colour:
        if target == 0:
            if cube[1][0][1] == cube[1][1][1] and cube[2][0][1] == cube[2][1][1] and cube[3][0][1] == cube[3][1][1] and cube[4][0][1] == cube[4][1][1]:
                return True
        elif target == 1:
            if cube[0][1][0] == cube[0][1][1] and cube[2][1][0] == cube[2][1][1] and cube[4][1][2] == cube[4][1][1] and cube[5][1][0] == cube[5][1][1]:
                return True
        elif target == 2:
            if cube[0][2][1] == cube[0][1][1] and cube[1][1][2] == cube[1][1][1] and cube[3][1][0] == cube[3][1][1] and cube[5][0][1] == cube[5][1][1]:
                return True
        elif target == 3:
            if cube[0][1][2] == cube[0][1][1] and cube[2][1][2] == cube[2][1][1] and cube[4][1][0] == cube[4][1][1] and cube[5][1][2] == cube[5][1][1]:
                return True
        elif target == 4:
            if cube[0][0][1] == cube[0][1][1] and cube[2][1][0] == cube[2][1][1] and cube[3][1][2] == cube[3][1][1] and cube[5][2][1] == cube[5][1][1]:
                return True
        elif target == 5:
            if cube[1][2][1] == cube[1][1][1] and cube[2][2][1] == cube[2][1][1] and cube[3][2][1] == cube[3][1][1] and cube[4][2][1] == cube[4][1][1]:
                return True
    return False